# Ajuste fino do conjunto de dados do Cohebert manual usando BERT Transformers by HuggingFace e Lotes Inteligentes e Validação Cruzada para todos os Folds

Com base no MCL BERT pré-treinado realiza o ajuste fino para os dados do Cohebert para classificar documentos originals e perturbados.

Classes:
- 1 - Documento original
- 0 - Documento perturbado

- Realiza o ajuste fino nos dados.
- Realiza a avaliação do conjunto de teste para um determinado fold(10%).
- A seção 2 - parametrização define os argumentos da execução.

Notebook base: https://colab.research.google.com/drive/1KDeFAHvRFq3bY5onzj8Wgul3rRcrQNaC?usp=sharing

Utiliza o arquivo `COHEBERT_MANUAL_KFOLD_10_PX_KY.zip`, X é o número de documentos perturbados e Y o valor de top K predições.

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

**Artigo padding dinâmico:**
https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e

# 1 Preparação do ambiente
Preparação do ambiente para execução do notebook.

## 1.1 Tempo inicial de processamento

In [ ]:
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

In [ ]:
print("  Tempo de início de processamento:  {:} (h:mm:ss)".format(inicio_processamento))

  Tempo de início de processamento:  1645800920.389201 (h:mm:ss)


## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """    
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
import time
import datetime

def formataTempo(tempo):
    '''
    Pega a tempo em segundos e retorna uma string hh:mm:ss
    '''
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Calcula a média de uma lista tempo string no formato hh:mm:ss.

In [ ]:
# Import das bibliotecas.
from cmath import rect, phase
from math import radians, degrees
  
def mediaAngulo(deg):
    return degrees(phase(sum(rect(1, radians(d)) for d in deg)/len(deg)))
 
def mediaTempo(tempos):
    '''
    Calcula a média de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Verifica se deu algum dia
    dia = 24 * 60 * 60
    # Converte para angulos
    para_angulos = [s * 360. / dia for s in segundos]
    # Calcula a média dos angulos
    media_como_angulo = mediaAngulo(para_angulos)
    media_segundos = media_como_angulo * dia / 360.
    if media_segundos < 0:
        media_segundos += dia
    # Recupera as horas e os minutos  
    h, m = divmod(media_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Calcula a soma de uma lista de tempo string no formato hh:mm:ss

In [ ]:
def somaTempo(tempos):
    '''
    Calcula a soma de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Soma os segundos
    soma_segundos = sum([s * 1. for s in segundos])
    # Recupera as horas e os minutos   
    h, m = divmod(soma_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)    
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Em muitos dos meus loops for (de longa duração), imprimirei atualizações periódicas de progresso. Normalmente, eu escolho o intervalo de atualização manualmente, mas para este Notebook, defini uma função auxiliar para fazer essa escolha para mim :)

In [ ]:
def obterIntervaloAtualizacao(total_iteracoes, numero_atualizacoes):
    '''
     Esta função tentará escolher um intervalo de atualização de progresso inteligente
     com base na magnitude das iterações totais.

     Parâmetros:
       `total_iteracoes` - O número de iterações no loop for.
       `numero_atualizacoes` - Quantas vezes queremos ver uma atualização sobre o
                               curso do loop for.
     '''
    
    # Divida o total de iterações pelo número desejado de atualizações. Provavelmente
    # este será um número feio.
    intervalo_exato = total_iteracoes / numero_atualizacoes

    # A função `arredondar` tem a capacidade de arredondar um número para, por exemplo, o
    # milésimo mais próximo: round (intervalo_exato, -3)
    #
    # Para determinar a magnitude para arredondar, encontre a magnitude do total,
    # e então vá uma magnitude abaixo disso.
    
    # Obtenha a ordem de magnitude do total.
    ordem_magnitude = len(str(total_iteracoes)) - 1
    
    # Nosso intervalo de atualização deve ser arredondado para uma ordem de magnitude menor.
    magnitude_arrendonda = ordem_magnitude - 1

    # Arredonde para baixo e lance para um int.
    intervalo_atualizacao = int(round(intervalo_exato, -magnitude_arrendonda))

    # Não permita que o intervalo seja zero!
    if intervalo_atualizacao == 0:
        intervalo_atualizacao = 1

    return intervalo_atualizacao

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )

Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo para classificação

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosClassificacao:
    '''
    Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo BERT para a classificação de coerência.
    '''
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    num_labels: int = field(
        default=2,
        metadata={"help": "número de rótulos a serem classificados."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    optimizer: str = field(
        default="AdamW",
        metadata={"help": "otimizador do modelo."},
    )
    use_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o uso do wandb."},
    )
    salvar_modelo_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do modelo no wandb."},
    )
    salvar_modelo : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento do modelo."},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do resultado da avaliação."},
    )     
    salvar_classificacao : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento da classificação."},
    )
    usar_mcl_ajustado: bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )    
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos perturbados comparados com o seu original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de previsões de palavras recuperadas mais próximas da máscara."},
    ) 
    epoca: int = field(
        default="1",
        metadata={"help": "Época a ser avaliada."},
    )    
    fold: int = field(
        default="1",
        metadata={"help": "Fold a ser avaliado."},
    )    

Biblioteca de limpeza de tela

In [ ]:
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Biblioteca de logging
import logging

# Formato da mensagem
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

2022-02-25 14:55:26,818 : INFO : Encontrei GPU em: /device:GPU:0


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.
    
    Retorno:
    `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
device = getDeviceGPU()

2022-02-25 14:55:33,250 : INFO : Existem 1 GPU(s) disponíveis.
2022-02-25 14:55:45,550 : INFO : Iremos usar a GPU: Tesla P100-PCIE-16GB.


Conecta o modelo ao device

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.       
        `device` - Um device de GPU.     
    
      Retorno:
        `model` - Um objeto model BERT conectado a GPU.     
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():    
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()
        
    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [ ]:
# Import das bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

2022-02-25 14:55:45,592 : INFO : Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

2022-02-25 14:55:45,594 : INFO : Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
2022-02-25 14:55:45,596 : INFO : e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.



In [ ]:
# Monta o Google Drive para esta instância de notebook.
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 11.4 MB/s 
     |████████████████████████████████| 181 kB 50.2 MB/s 
     |████████████████████████████████| 144 kB 51.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=876a91b40e2803be285203f1920e6a09101b2e7b152928103a1ca92148aa94c1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


Login via linha de comando

In [ ]:
!wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.8 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1 MB 12.3 MB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 


# 2 Parametrização

## Gerais

In [ ]:
# Prefixo do nome do arquivo usado nas saídas projeto C = Cris, SB = SmartBatch, KFT = KFold
NOME_BASE_SAIDA = "AjusteFinoCohebertManual_C_SB_KF_v1"

# Definição dos parâmetros a serem avaliados

######## Parâmetros Individuais ########

# Quantidade de documentos a serem perturbados a partir do original.    
DOCUMENTOS_PERTURBADOS = 1 # Somente 1 para o Cohebert manual

# Quantidade de palavras a serem recuperadas mais próximas da máscara. 
TOP_K_PREDICAO = 1 # Somente 1 para o Cohebert manual

# Tamanho dos lotes de treino e avaliação. Usar 16 ou 32
TAMANHO_LOTE = 16

######## Parâmetros de conjunto ########
# Taxas de aprendizagem a serem avaliados
TAXAS_DE_APRENDIZAGEM = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5]

# MCL a serem avaliados
#NOMES_MODELO = ['https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip',
#                'https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip',
#                'bert-base-multilingual-cased']

NOMES_MODELO = ["neuralmind/bert-large-portuguese-cased"]

######## Parâmetros de intervalo ########
# Número de épocas a serem avaliadas
# Todas as épocas são avaliadas e os resultados da classificação são salvos. De 1 até 4
EPOCAS = 4

# Determina o intervalo de folds a ser avaliado. De 1 até 10
# Usado para quando o processamento é interrompido
inicio_fold = 1
fim_fold = 10

## Específicos

Parâmetros de treinamento

In [ ]:
# Importando as bibliotecas.
from transformers import TrainingArguments

# Definição dos parâmetros de Treinamento
training_args = TrainingArguments(
    # NOME_BASE_SAIDA = Nome base do arquivo de saída
    # P = documentos perturbados
    # K = previsões palavras 
    # E = número total de épocas de treinamento
    # e = número da época executada
    # lr = taxa de aprendizagem
    # b = lotes de treino e avaliação    
    # f = número do fold
    output_dir = NOME_BASE_SAIDA + "K_1_P_1_E_4_e_1_lr_5_b_8_4_f", # É utilizado somente para logs de arquivo e wandb   
    save_steps = 0,    
    seed = 42,
    num_train_epochs = EPOCAS, # Quantidade de épocas de treinamento
    learning_rate = 5e-5, # Intervalo de valores: 1e-5, 2e-5, 3e-5, 4e-5, 5e-5. É utilizado somente para logs.
    gradient_accumulation_steps = 1,
    per_device_train_batch_size = TAMANHO_LOTE, 
    per_device_eval_batch_size = TAMANHO_LOTE,        
    evaluation_strategy = 'epoch',
)

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosClassificacao(     
    max_seq_len = 512,
    # É utilizado somente para logs
    pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",   
    do_lower_case = False, # default True
    num_labels = 2,
    output_attentions = False, # default False
    output_hidden_states = False, # default False 
    optimizer = 'AdamW',
    use_wandb = True, # Ativa a gravação de logs no wandb
    salvar_modelo_wandb = False, # Ativa o salvamento do MCL no wandb
    salvar_modelo = False, # Ativa o salvamento do MCL no googledrive
    salvar_avaliacao = True, # Salva o resultado classificações
    salvar_classificacao = True, # Salva o resultado da avaliação das classificações
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.    
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara. 
    fold = 1 # Intervalo de valores: 1 a 10, É utilizado somente para logs. Use as variáveis do bloco a seguir para definir um intervalo dos folds.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHEBERT_MANUAL"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):  
      # Cria o diretório
      os.makedirs(dirbase)    
      logging.info("Diretório criado: {}.".format(dirbase))
  else:    
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

2022-02-25 14:56:28,759 : INFO : Diretório criado: /content/COHEBERT_MANUAL.


Diretório para conter as os resultados das classificações

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "/validacao_classificacao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacao()

2022-02-25 14:56:29,411 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL//validacao_classificacao_palavra.


Diretório para conter os arquivos da avaliação kfold

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacaoKfold():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_classificacao_palavra/kfold"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacaoKfold()

2022-02-25 14:56:29,767 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold.


Diretório para conter os arquivos de classificação da avaliação kfold

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacaoKfoldClassificacao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_classificacao_palavra/kfold/Classificacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacaoKfoldClassificacao()

2022-02-25 14:56:30,105 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold/Classificacao.


Diretório para conter os arquivos de resultado da avaliação kfold

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacaoKfoldAvaliacao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_classificacao_palavra/kfold/Avaliacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):  
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacaoKfoldAvaliacao()

2022-02-25 14:56:30,141 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold/Avaliacao.


# 3 BERT

## 3.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''    
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"
    
    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"
        
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
            
    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''    
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.       
    
    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''
    
    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"
    
    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"
        
    return TAMANHO_BERT  

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 
    
    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO
        
        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"
        
        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo.")
            # Apaga o diretório e os arquivos existentes                     
            shutil.rmtree(DIRETORIO_MODELO)
        
        # Realiza o download do arquivo do modelo        
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.                
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.        
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)
        
        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta.".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada.")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """ 
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    
      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO) 
   
    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """ 
    Verifica de onde utilizar o modelo.
    
    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """ 

    DIRETORIO_MODELO = None
    
    if model_args.usar_mcl_ajustado == True:        
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()
        
        logging.info("Usando modelo BERT ajustado.")
        
    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")        
        
    return DIRETORIO_MODELO

## 3.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).



In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.       
    
      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None
    
    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 3.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

### Função carrega modelo BERT medida




In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.
    
      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
        `model_args` - Objeto com os argumentos do modelo.   
    
      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:        
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)
        
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega modelo BERT classificação



In [ ]:
# Import das bibliotecas.
from transformers import BertForSequenceClassification # Importando as bibliotecas do Modelo BERT.

def carregaModeloClassifica(DIRETORIO_MODELO, model_args):
    ''' 
    Carrega o modelo e retorna o modelo.
    
    Parâmetros:
    `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.           
    `model_args` - Objeto com os argumentos do modelo.
    
    Retorno:
    `model` - Um objeto do modelo BERT carregado.
    ''' 

    # Variável para setar o arquivo.
    URL_MODELO = None

    if 'http' in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para classificação.".format(DIRETORIO_MODELO))

        model = BertForSequenceClassification.from_pretrained(DIRETORIO_MODELO, 
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)
            
    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para classificação.".format(model_args.pretrained_model_name_or_path))

        model = BertForSequenceClassification.from_pretrained(model_args.pretrained_model_name_or_path,
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)
    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """ 
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:    
        `model` - Um objeto do modelo BERT carregado.       
        `tokenizer` - Um objeto tokenizador BERT carregado.       
    """
            
    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)
    
    # Variável para conter o modelo
    model = None
    
    # Verifica o tipo do modelo em model_args    
    if type(model_args) == ModeloArgumentosMedida:
        # Carrega o modelo para cálculo da medida
        model = carregaModeloMedida(DIRETORIO_MODELO, model_args)
        
    else:
        # Carrega o modelo para classificação
        model = carregaModeloClassifica(DIRETORIO_MODELO, model_args)
        
        # Recupera o dispositivo da GPU 
        device = getDeviceGPU()
    
        # Conecta o modelo a GPU
        model = conectaGPU(model, device)
                       
    # Carrega o tokenizador. 
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)
    
    return model, tokenizer

### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

# 4 Treinamento e avaliação

## 4.1 Wandb

https://wandb.ai/osmar-braz/AjusteFinoCohebertManual_v1_C_SB_KF/table?workspace=user-osmar-braz

https://wandb.ai/osmar-braz/AjusteFinoCohebertManual_v2_C_SB_KF/table?workspace=user-osmar-braz

### Função de inicialização wandb

In [ ]:
# Importando a biblioteca.
import wandb

def inicializacaoWandb():

  if model_args.use_wandb:

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    # O projeto no wandb recebe o nome base mais o número de um fold especifico.
    wandb.init(project=NOME_BASE_SAIDA, name=training_args.output_dir + str(model_args.fold))
    
    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)
    # Atualiza os parâmetros de treinamento no wandb.
    wandb.config.update(training_args)
    wandb.config.dataset = DIRETORIO_COHEBERT
    wandb.config.batch_size = training_args.per_device_train_batch_size
    
    # Registra os parämetros não literais do model_args.    
    wandb.log({"max_seq_len": model_args.max_seq_len})
    wandb.log({"do_lower_case": model_args.do_lower_case})
    wandb.log({"output_hidden_states": model_args.output_hidden_states})

    return wandb

## 4.2 Carregamento dos arquivos de dados kfold

### Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_FOLD_COMPACTADO = DIRETORIO_COHEBERT + "_KFOLD_10" + "_P" +  str(model_args.documentos_perturbados) + "_K" + str(model_args.top_k_predicao) + ".zip"

### Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
def copiaArquivoFold():
  
  # Se estiver executando no Google Colaboratory
  if IN_COLAB:

    criaDiretorioLocal()

    !cp '$DIRETORIO_DRIVE''/validacao_classificacao_palavra/kfold/'$NOME_ARQUIVO_FOLD_COMPACTADO $DIRETORIO_LOCAL
      
    logging.info("Terminei a cópia.")

In [ ]:
copiaArquivoFold()

2022-02-25 14:56:31,165 : INFO : Diretório já existe: /content/COHEBERT_MANUAL.
2022-02-25 14:56:31,762 : INFO : Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [ ]:
def descompactaArquivoFold():
  
  # Se estiver executando no Google Colaboratory
  if IN_COLAB:
    !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_FOLD_COMPACTADO" -d "/content/validacao_kfold"

    logging.info("Terminei a descompactação.")

In [ ]:
descompactaArquivoFold()

2022-02-25 14:56:32,024 : INFO : Terminei a descompactação.


### Função de carregamento dos dados de um fold

In [ ]:
# Import das bibliotecas.
import pandas as pd

def carregamentoDadosFold(fold):

  logging.info("Carregando os arquivos do fold de treino e avaliação.")
  
  # Diretório dos arquivos de dados.
  DIRETORIO = "/content/validacao_kfold"

  # Define o prefixo do nome dos arquivos dos folds
  PREFIXO_NOME_ARQUIVO_TREINO = DIRETORIO_COHEBERT + "_Train_f"
  PREFIXO_NOME_ARQUIVO_TESTE = DIRETORIO_COHEBERT + "_Test_f"

  # Nome dos arquivos.
  ARQUIVO_TREINO = DIRETORIO + "/" + PREFIXO_NOME_ARQUIVO_TREINO + str(fold) + ".csv"
  ARQUIVO_TESTE = DIRETORIO + "/" + PREFIXO_NOME_ARQUIVO_TESTE + str(fold) + ".csv" 

  logging.info("Carregando arquivo de treino: {}.".format(ARQUIVO_TREINO))
  logging.info("Carregando arquivo de teste: {}.".format(ARQUIVO_TESTE))

  # Verifica se o arquivo existe
  if not os.path.isfile(ARQUIVO_TREINO):
      # Caso contrário copia o arquivo do drive e descompacta
      copiaArquivoFold()
      descompactaArquivoFold()

  # Carrega o conjunto de dados de treino e teste.
  df_dados_train = pd.read_csv(ARQUIVO_TREINO, sep=';')  
  df_dados_test = pd.read_csv(ARQUIVO_TESTE, sep=';')  
  #logging.info("Qtde de dados de treino: {} e teste {}.".format(len(df_dados_train), len(df_dados_test)))

  return df_dados_train, df_dados_test

## 4.3 Análise e tratamento dos dados

### Função descarte documentos muito grandes

In [ ]:
def descarteDocumentosGrandes(tokenizer, 
                              tamanho_maximo_token, 
                              df_dados_train, 
                              df_dados_test):

  logging.info("Descartando documentos grandes dos conjuntos de dados.")

  # Define o tamanho máximo para os tokens.
  tamanho_maximo = tamanho_maximo_token

  # Tokenize a codifica as setenças para o BERT.     
  df_dados_train["input_ids"] = df_dados_train["documento"].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))
        
  df_dados_train = df_dados_train[df_dados_train["input_ids"].apply(len)<tamanho_maximo]

  # Remove as colunas desnecessárias.
  df_dados_train = df_dados_train.drop(columns=["input_ids"])

  # Tokenize a codifica as setenças para o BERT.     
  df_dados_test["input_ids"] = df_dados_test["documento"].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))

  # Corta os inputs para o tamanho máximo 512.
  df_dados_test = df_dados_test[df_dados_test["input_ids"].apply(len)<tamanho_maximo]

  #logging.info("Tamanho do conjunto de dados: {} / Treino: {} / Teste: {}.".format((len(df_dados_train)+len(df_dados_test)),len(df_dados_train), len(df_dados_test)))
    
  # Remove as colunas desnecessárias.
  df_dados_test = df_dados_test.drop(columns=["input_ids"])

  del tokenizer
  del tamanho_maximo

  return df_dados_train, df_dados_test

## 4.4 Treinamento do modelo 

### Função do Otimizador e Agendador de Taxas de Aprendizado/Optimizer & Learning Rate Scheduler



Agora que temos nosso modelo carregado, precisamos pegar os hiperparâmetros de treinamento no modelo armazenado.

Para fins de ajuste fino, os autores recomendam escolher entre os seguintes valores (no Apêndice A.3 do [artigo BERT](https://arxiv.org/pdf/1810.04805.pdf)):

> - **Tamanho do lote(Batch size):** 16, 32
- **Taxa de aprendizado (Adam):** 5e-5, 3e-5, 2e-5
- **Número de épocas:** 2, 3, 4

O parâmetro epsilon `eps = 1e-6` é" um número muito pequeno para impedir qualquer divisão por zero na implementação "(a partir de [aqui](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).

Você pode encontrar a criação do otimizador do AdamW em `run_glue.py` [aqui](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109).

### Função carrega otimizador


In [ ]:
# Import das bibliotecas.
from transformers import AdamW

def carregaOtimizador(training_args, model):
  '''
    Esta função carrega o otimizador utilizado no agendador de aprendizado.
  '''

  # Nota: AdamW é uma classe da biblioteca huggingface (ao contrário de pytorch).
  # Eu acredito que o 'W' significa 'Correção de redução de peso "
  optimizer = AdamW(model.parameters(),
                  lr = training_args.learning_rate, # (ou alfa) A taxa de aprendizado a ser usada. - default é 3e-5
                  # betas = (0.9, 0.999), # (beta1, beta2) - default é (0.9, 0.999)
                    # beta1 é taxa de decaimento exponencial para as estimativas do primeiro momento. 
                    # beta2 é taxa de decaimento exponencial para as estimativas do segundo momento. Este valor deve ser definido próximo a 1,0 em problemas com gradiente esparso (por exemplo, PNL e problemas de visão de computacional)
                  # eps = 1e-6, #  É um número muito pequeno para evitar qualquer divisão por zero na implementação - default é 1e-6.
                  # weight_decay = 0.0, # Correção de redução de peso. - default é 0.0
                    # A redução da taxa de aprendizagem também pode ser usada com Adam. A taxa de decaimento é atualizada a cada época para a demonstração da regressão logística.
                  # correct_bias = True #  Se não deve corrigir o viés(bias) no Adam mudar para False.- default é True
                )
  
  return optimizer

### Função carrega agendador

A função **get_linear_schedule_with_warmup** cria um agendador com uma taxa de aprendizado que diminua linearmente da taxa de aprendizagem inicial definido no otimizador até 0, após um período de aquecimento durante o qual ele aumenta linearmente de 0 para a taxa de aprendizagem inicial definido no otimizador.

Se `num_warmup_steps=0` e `weight_decay=0`(otimizador) não ocorre a etapa de aquecimento.

In [ ]:
# Import das bibliotecas.
from transformers import get_linear_schedule_with_warmup

def carregaAgendador(training_args, optimizer, numero_etapas):
  '''
    Esta função carrega o agendador com um taxa de aprendizado que diminua linearmente até 0.
  '''

  # O número total de etapas de ajuste fino é [número de lotes] x [número de épocas].
  # (Observe que este não é o mesmo que o número de amostras de ajuste fino).
  total_etapas = numero_etapas * training_args.num_train_epochs

  #Cria o agendador de taxa de aprendizagem.
  scheduler = get_linear_schedule_with_warmup(optimizer, # O otimizador para o qual agendar a taxa de aprendizado.
                                            num_warmup_steps = 0, # O número de etapas para a fase de aquecimento. Valor default value em run_glue.py
                                            num_training_steps = total_etapas) # O número total de etapas de treinamento.


  logging.info("Total de etapas: {}".format(total_etapas))

  return scheduler

### Função cria lotes inteligentes

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook
import random

def criarLotesInteligentes(tokenizer, 
                            documentos, 
                            classes, 
                            id_documentos, 
                            batch_size):
    '''
    Esta função combina todos os passos para preparar os lotes.
    '''
    logging.info("Criando Lotes Inteligentes de {:,} amostras com tamanho de lote {:,}...".format(len(documentos), batch_size))

    # ============================
    #   Tokenização & Truncamento
    # ============================

    input_ids_completos = []
    
    # Tokeniza todas as amostras de treinamento
    #logging.info("Tokenizando {:,} amostra...".format(len(classes)))
    
    # Escolha o intervalo que o progresso será atualizado.
    #intervalo_atualizacao = obterIntervaloAtualizacao(total_iteracoes=len(classes), numero_atualizacoes=10)
    
    # Barra de progresso dos documentos
    #documentos_bar = tqdm_notebook(documentos, desc=f'Documentos ', unit=f'documento', total=len(documentos))

    # Para cada amostra de treinamento...
    #for documento in documentos_bar:
    for documento in documentos:
    
        # Relatório de progresso
        #if ((len(input_ids_completos) % intervalo_atualizacao) == 0):
        #    logging.info("  Tokenizado {:,} amostras.".format(len(input_ids_completos)))

        # Tokeniza a amostra.
        input_ids = tokenizer.encode(text=documento,                    # Documento a ser codificado.
                                    add_special_tokens=True,            # Adiciona os ttokens especiais.
                                    max_length=model_args.max_seq_len,  # Tamanho do truncamento!
                                    truncation=True,                    # Faz o truncamento!
                                    padding=False)                      # Não preenche.
                
        # Adicione o resultado tokenizado à nossa lista.
        input_ids_completos.append(input_ids)

        del input_ids
    
    del documentos    
    
    #logging.info("{:>10,} amostras tokenizadas.".format(len(input_ids_completos)))

    # =========================
    #      Seleciona os Lotes
    # =========================    
    
    # Classifique as duas listas pelo comprimento da sequência de entrada.
    amostras = sorted(zip(input_ids_completos, classes, id_documentos), key=lambda x: len(x[0]))

    del input_ids_completos
    del classes
    del id_documentos

    #logging.info("{:>10,} amostras após classificação.".format(len(amostras)))

    # Lista de lotes que iremos construir.
    batch_ordered_documentos = []
    batch_ordered_classes = []
    batch_ordered_id_documentos = []

    logging.info("Criando lotes de tamanho {:}...".format(batch_size))

    # Escolha um intervalo no qual imprimir atualizações de progresso.
    intervalo_atualizacao = obterIntervaloAtualizacao(total_iteracoes=len(amostras), numero_atualizacoes=10)
        
    # Faça um loop em todas as amostras de entrada ... 
    while len(amostras) > 0:
        
        # Mostra o progresso.
        # if ((len(batch_ordered_documentos) % intervalo_atualizacao) == 0 \          
          #  and not len(batch_ordered_documentos) == 0):
           #logging.info("  Selecionado {:,} lotes.".format(len(batch_ordered_documentos)))
        
        # `to_take` é o tamanho real do nosso lote. Será `batch_size` até
        # chegamos ao último lote, que pode ser menor.
        to_take = min(batch_size, len(amostras))
        
        # Escolha um índice aleatório na lista de amostras restantes para começar o nosso lote.
        select = random.randint(0, len(amostras) - to_take)

        # Selecione um lote contíguo de amostras começando em `select`.
        #print ("Selecionando lote de {:} a {:}".format(select, select+to_take))
        batch = amostras[select:(select + to_take)]

        #print("Tamanho do lote:", len(batch))
        
        # Cada amostra é uma tupla --divida para criar uma lista separada de
        # sequências e uma lista de rótulos para este lote.
        batch_ordered_documentos.append([s[0] for s in batch])
        batch_ordered_classes.append([s[1] for s in batch])
        batch_ordered_id_documentos.append([s[2] for s in batch])
        
        # Remova a amostra da lista
        del amostras[select:select + to_take]

    #logging.info("  FEITO - Selecionado {:,} lotes.".format(len(batch_ordered_documentos)))

    # =========================
    #        Adicionando o preenchimento
    # =========================    

    #logging.info("Preenchendo sequências dentro de cada lote...")

    py_input_ids = []
    py_attention_masks = []
    py_labels = []
    list_id_documentos = []

    # Para cada lote...
    for (batch_input_ids, batch_labels, batch_id_documentos) in zip(batch_ordered_documentos, batch_ordered_classes, batch_ordered_id_documentos):

        # Nova versão do lote, desta vez com sequências preenchidas e agora com
        # as máscaras de atenção definidas.
        batch_padded_input_ids = []
        batch_attention_masks = []
                
        # Primeiro, encontre a amostra mais longa do lote.
        # Observe que as sequências atualmente incluem os tokens especiais!
        max_size = max([len(input) for input in batch_input_ids])
        
        # Para cada entrada neste lote...
        for input in batch_input_ids:
                        
            # Quantos tokens pad precisam ser adicionados
            num_pads = max_size - len(input)

            # Adiciona `num_pads` do pad token(tokenizer.pad_token_id) até o final da sequência.
            padded_input = input + [tokenizer.pad_token_id] * num_pads

            # Define a máscara de atenção --é apenas um `1` para cada token real
            # e um `0` para cada token de preenchimento(pad).
            attention_mask = [1] * len(input) + [0] * num_pads
            
            # Adiciona o resultado preenchido ao lote.
            batch_padded_input_ids.append(padded_input)
            batch_attention_masks.append(attention_mask)

            del padded_input
            del attention_mask
        
        # Nosso lote foi preenchido, portanto, precisamos salvar este lote atualizado.
        # Também precisamos que as entradas sejam tensores PyTorch, então faremos isso aqui.
        py_input_ids.append(torch.tensor(batch_padded_input_ids))
        py_attention_masks.append(torch.tensor(batch_attention_masks))
        py_labels.append(torch.tensor(batch_labels))
        list_id_documentos.append(batch_id_documentos)
        
        del batch_padded_input_ids
        del batch_attention_masks

    del batch_ordered_documentos
    del batch_ordered_classes
    del batch_ordered_id_documentos
    
    del tokenizer    

    # Retorna o conjunto de dados em lotes inteligentes!
    return (py_input_ids, py_attention_masks, py_labels, list_id_documentos)

### Função de Treinamento

In [ ]:
# Import das bibliotecas
import random
import numpy as np
from tqdm.notebook import tqdm as tqdm_notebook

def realizaTreinamento(model, 
                       tokenizer, 
                       optimizer, 
                       scheduler,  
                       documentos_treino, 
                       classes_treino, 
                       id_documentos_treino, 
                       documentos_teste, 
                       classes_teste, 
                       id_documentos_teste, 
                       EPOCAS = 4):
  
  #logging.info("Realizando treinamento e avaliação do fold: {}".format(model_args.fold))

  # Defina o valor da semente em todos os lugares para torná-lo reproduzível.
  seed_val = training_args.seed
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # Medida do tempo total de treinamento e avaliação.
  treinamento_avaliacao_t0 = time.time()

  # Limpa o cache da GPU.
  torch.cuda.empty_cache()

  # Coloque o modelo em modo de treinamento. 
  model.train()

  # Acumula as perdas do treinamento.
  train_losses = []
  test_losses = []
   
  # Barra de progresso da época.
  epoca_bar = tqdm_notebook(range(1,training_args.num_train_epochs+1), desc=f'Épocas', unit=f'épocas')
    
  # Para cada época.
  for epoca_i in epoca_bar:
    
    # ========================================
    #               Treinamento e Avaliação
    # ========================================
    
    # Atualiza a época avaliada
    model_args.epoca = epoca_i

    # Atualiza o nome da saída avaliada
    training_args.output_dir = NOME_BASE_SAIDA + getSufixoNomeArquivoSaida(training_args, model_args)

    # Inicializa o wandb para registro
    # Gera uma entrada para cada todas as epocas, com a taxa de aprendizagem, lote e fold    
    wandb = inicializacaoWandb()
    
    # Log das métricas com wandb.
    if model_args.use_wandb:    
      wandb.watch(model)  
    
    # Execute uma passada completa sobre o conjunto de treinamento.

    logging.info("Realizando treinamento do fold {} na época: {}.".format(model_args.fold, model_args.epoca))

    # Recupera o lote inteligente
    (py_input_ids, py_attention_masks, py_labels, id_documentos) = criarLotesInteligentes(tokenizer, documentos_treino, classes_treino, id_documentos_treino, training_args.per_device_train_batch_size)
    
    # Medida de quanto tempo leva o período de treinamento.
    treinamento_epoca_t0 = time.time()

    # Acumula as perdas do treinamento da época.
    train_batch_losses = []

    # Barras de progresso.    
    lote_treino_bar = tqdm_notebook(range(0, len(py_input_ids)), desc=f'Epoca {epoca_i}', unit=f'lotes', total=len(py_input_ids) )

    # Para cada lote dos dados de treinamento.
    for index in lote_treino_bar:      

        # Descompacte este lote de treinamento de nosso dataloader.
        #
        # À medida que descompactamos o lote, também copiaremos cada tensor para a GPU usando o
        # o método `to`
        #
        # `lote` é uma lista contém três tensores pytorch:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        # Recupera os tensores do lote e copia para a GPU usando o método `to` 
        d_input_ids = py_input_ids[index].to(device)
        d_input_mask = py_attention_masks[index].to(device)
        d_labels = py_labels[index].to(device)     
        
        # Sempre limpe quaisquer gradientes calculados anteriormente antes de realizar um
        # passe para trás. PyTorch não faz isso automaticamente porque
        # acumular os gradientes é "conveniente durante o treinamento de RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Execute um passe para frente (avalie o modelo neste lote de treinamento).
        # A documentação para esta função `model` está aqui:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Ele retorna diferentes números de parâmetros dependendo de quais argumentos
        # são fornecidos e quais sinalizadores estão definidos. Para nosso uso aqui, ele retorna
        # a perda (porque fornecemos rótulos) e os "logits" - o modelo de saídas antes da ativação.     

        # last_hidden_state = outputs[0], pooler_output = outputs[1], hidden_states = outputs[2]
        outputs = model(d_input_ids, 
                        token_type_ids=None, 
                        attention_mask=d_input_mask, 
                        labels=d_labels)
        
        # A perda(loss) é retornado em outputs[0] porque fornecemos rótulos(labels))                  
        loss = outputs[0]

        # E outputs[1] os "logits" - o modelo de saídas antes da ativação.
        # logits possui duas dimensões, a primeira do lote e a segunda do rótulo da predição                        
        # A função `.detach().cpu()` retira da gpu.
        logits = outputs[1].detach().cpu()
  
        # Acumule a perda de treinamento em todos os lotes da época para que possamos
        # calcular a perda média no final da época. `loss` é um tensor contendo um único valor.   
        # A função `.item ()` retorna apenas o valor Python do tensor.
        train_batch_losses.append(loss.item())

        # Mostra a perda na barra de progresso.
        lote_treino_bar.set_postfix(loss=loss.item())

        # Log das métricas com wandb.
        if model_args.use_wandb:          
          wandb.log({"train_batch_loss": loss.item()})

        # Execute uma passagem para trás para calcular os gradientes.
        # Todos os parâmetros do modelo deve ter sido setado para param.requires_grad = False
        loss.backward()            

        # Corte a norma dos gradientes para 1.0.
        # Isso ajuda a evitar o problema de "gradientes explosivos".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
       
        # Atualize os parâmetros e dê um passo usando o gradiente calculado.
        # O otimizador dita a "regra de atualização" - como os parâmetros são
        # modificados com base em seus gradientes, taxa de aprendizagem, etc.
        optimizer.step()
                           
        # Atualize a taxa de aprendizagem.
        scheduler.step()

        # Apaga variáveis não utilizadas
        del outputs

    # Média da perda do treinamento de todos os lotes da época.
    media_train_epoca_loss = np.mean(train_batch_losses)

    # Acumule a perda de treinamento de todas as épocas para calcular a perda média do treinamento.    
    train_losses.append(media_train_epoca_loss)

    # Medida de quanto tempo levou o treinamento desta época.
    treinamento_epoca_total = formataTempo(time.time() - treinamento_epoca_t0)

    logging.info("   Média perda(loss) do treinamento da época : {0:.8f}.".format(media_train_epoca_loss))
    logging.info("   Tempo de treinamento da época             : {:}.".format(treinamento_epoca_total))    
    logging.info("   Tempo parcial processamento               : {:} (h:mm:ss)".format(formataTempo(time.time()-treinamento_avaliacao_t0)))

    ################# Avaliação da época
    
    # Registra o tempo inicial.
    avaliacao_epoca_t0 = time.time()

    # Realiza a avaliação do modelo.    
    media_test_epoca_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, lista_resultado_avaliacao = realizaAvaliacao(epoca_i, model, tokenizer, optimizer, scheduler, wandb, documentos_teste, classes_teste, id_documentos_teste)

    logging.info("   Avaliação loss: {:.8f}; Acc: {:.8f}; Rec: {:.8f}; Pre: {:.8f}, F1:{:.8f}, vp: {:3d}; vn: {:3d}; fp: {:3d}; fn: {:3d}".format(media_test_epoca_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s))      
    logging.info("   Acurácia do fold {} na época {}: {:.8f}.".format(model_args.fold, epoca_i, acc))  
    
    # Acumule a perda de treinamento da época para calcular a perda média do treinamento.    
    test_losses.append(media_test_epoca_loss)

    # Medida de quanto tempo levou a execução da avaliação
    avaliacao_epoca_total = formataTempo(time.time() - avaliacao_epoca_t0)

    logging.info("   Média perda(loss) do avaliação da época   : {0:.8f}.".format(media_test_epoca_loss))
    logging.info("   Tempo de avaliação da época               : {:}.".format(avaliacao_epoca_total))    
    logging.info("   Tempo parcial do processamento            : {:} (h:mm:ss)".format(formataTempo(time.time()-treinamento_avaliacao_t0)))

    ################# Salva a classificação e avaliação para a época

    # Salva o resultado da classificação da época
    salvaResultadoClassificacao(lista_resultado_avaliacao)

    # Salva o resultado da avaliação da épóca
    treinamento_avaliacao_total_t0 = format(formataTempo(time.time()-treinamento_avaliacao_t0))
    salvaResultadoAvaliacao(media_test_epoca_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, treinamento_avaliacao_total_t0)

    ################# Salva o modelo ajustado no wandb
    salvaModeloWandb(model, model_args)

    ################# Salva o modelo ajustado
    salvaModelo(model, tokenizer, model_args)

    # Apaga variáveis não utilizadas
    del py_input_ids
    del py_attention_masks
    del py_labels
    del train_batch_losses
    del lote_treino_bar

    # Log das métricas com wandb.
    if model_args.use_wandb:    
      NOME_EXECUCAO = NOME_BASE_SAIDA + getSufixoNomeArquivoSaida(training_args, model_args)
      wandb.log({"nome_execucao": NOME_EXECUCAO[:-2],  # -2 para retirar "_f" do final do nome da execução
                 "acuracia": acc,
                 "vp": vp_s , 
                 "vn": vn_s,  
                 "fp": fp_s,
                 "fn": fn_s, 
                 "media_train_epoca_loss" : media_train_epoca_loss,
                 "tempo_train" : treinamento_epoca_total,
                 "media_test_epoca_loss" : media_test_epoca_loss,
                 "tempo_test" : avaliacao_epoca_total})
      
      # Finaliza o wandb
      if model_args.use_wandb:
        wandb.finish()  
     
  # Média da perda do treinamento de todas as épocas.
  media_train_loss = np.mean(train_losses)
  media_test_loss = np.mean(test_losses)

  #if model_args.use_wandb:
  #  wandb.log({"media_train_loss": media_train_loss}, step = epoca_i)   
  #  wandb.log({"media_test_loss": media_test_loss}, step = epoca_i)

  logging.info("  Média perda(loss) treinamento : {0:.8f}.".format(media_train_loss))
  logging.info("  Média perda(loss) avaliação   : {0:.8f}.".format(media_test_loss))

  # Apaga variáveis não utilizadas
  del train_losses
  del epoca_bar
  
  del model
  del tokenizer  
  del documentos_treino
  del classes_treino
  del id_documentos_treino
  del documentos_teste
  del classes_teste
  del id_documentos_teste

## 4.5 Avaliação

Avaliando o modelo treinado no conjunto de dados de teste.

### Função de Avaliação

In [ ]:
# Import das bibliotecas.
import torch
from tqdm.notebook import tqdm as tqdm_notebook

def realizaAvaliacao(epoca, 
                     model, 
                     tokenizer, 
                     optimizer, 
                     scheduler, 
                     wandb,
                     documentos_teste, 
                     classes_teste, 
                     id_documentos_teste):
   
  # Armazena o resultado da avaliação executada
  lista_resultado_avaliacao = []

  logging.info("Realizando avaliação do fold {} na época: {}.".format(model_args.fold, epoca))

  # Use nossa nova função para preparar completamente nosso conjunto de dados.
  (py_input_ids, py_attention_masks, py_labels, documentosids) = criarLotesInteligentes(tokenizer, documentos_teste, classes_teste, id_documentos_teste, training_args.per_device_eval_batch_size)

  # Coloque o modelo em modo de avaliação.
  model.eval()

  # Acumula as perdas dos testes dos lotes.
  test_batch_losses = []

  # Acumula os resultados dos testes.
  vp = [] # Verdadeiro positivo
  vn = [] # Verdadeiro negativo
  fp = [] # Falso positivo
  fn = [] # Falso negativo

  # Barra de progresso dos lotes de teste.
  lote_teste_bar = tqdm_notebook(range(0, len(py_input_ids)), desc=f'Lotes ', unit=f'lotes', total=len(py_input_ids))

  # Para cada lote dos dados de avaliação(teste).
  for index in lote_teste_bar:

    # Copia o lote para a GPU.
    d_input_ids = py_input_ids[index].to(device)
    d_input_mask = py_attention_masks[index].to(device)
    d_labels = py_labels[index].to(device)
    d_id_documentos = documentosids[index]

    # Diga a pytorch para não se preocupar em construir o gráfico de computação durante
    # o passe para frente, já que isso só é necessário para backprop (treinamento).
    with torch.no_grad():
        # Obtenha a saída de "logits" pelo modelo. Os "logits" são a saída
        # valores antes de aplicar uma função de ativação como o softmax.        
        # Retorno de model quando ´last_hidden_state=True´ é setado:    
        # last_hidden_state = outputs[0], pooler_output = outputs[1], hidden_states = outputs[2]
        outputs = model(d_input_ids,
                        token_type_ids=None, 
                        attention_mask=d_input_mask, 
                        labels=d_labels)
        
    # A perda(loss) é retornado em outputs[0] porque fornecemos rótulos(labels). 
    # É útil para comparar com a perda do treinamento, quando é realizado a avaliação entre as épocas de treinamento.
    loss = outputs[0]

    # E outputs[1] os "logits" - o modelo de saídas antes da ativação.
    # logits possui duas dimensões, a primeira do lote e a segunda do rótulo da predição                        
    logits = outputs[1]
        
    # Acumule a perda da avaliação em todos os lotes para que possamos
    # calcular a perda média no final. `loss` é um tensor contendo um único valor.
    # A função '.cpu()' move loss para a cpu.
    # A função `.item ()` retorna apenas o valor Python do tensor.         
    test_batch_losses.append(loss.cpu().item())

    # Log das métricas com wandb.
    if model_args.use_wandb:
        wandb.log({"test_batch_loss": loss.cpu().item()})

    # Recupera o índice do melhor resultado, maior valor dos tensores para coluna(1)
    _, classificacao = torch.max(logits, 1)

    # Verifica a classificação realizada e o rótulo previsto
    vp.append(((classificacao==1) & (d_labels==1)).sum().cpu().item())
    vn.append(((classificacao==0) & (d_labels==0)).sum().cpu().item())
    fp.append(((classificacao==1) & (d_labels==0)).sum().cpu().item())
    fn.append(((classificacao==0) & (d_labels==1)).sum().cpu().item())

    # Adiciona o documento de teste, o rótulo e a classificação realizada a lista de resultado
    for lote in range(len(d_labels)):
                
        lista_resultado_avaliacao.append([d_id_documentos[lote],
                                d_labels[lote].cpu().item(), 
                                classificacao[lote].cpu().item()])

    del outputs
    del d_input_ids
    del d_input_mask
    del d_labels
    del d_id_documentos

  # Soma as classificações realizadas
  vp_s, vn_s, fp_s, fn_s = sum(vp), sum(vn), sum(fp), sum(fn)
  
  # Acurácia indica uma performance geral do modelo. 
  # Dentre todas as classificações, quantas o modelo classificou corretamente(vp=1 e vn=0).
  if (vp_s+vn_s+fp_s+fn_s) != 0:
      acc = (vp_s+vn_s)/(vp_s+vn_s+fp_s+fn_s)
  else:
      acc = 0

  # Recall(Revocação) avalia todas as situações da classe Positivo(vp=1) com o valor esperado e quantas estão corretas.
  if (vp_s+fn_s) != 0:
      rec = (vp_s)/(vp_s+fn_s)
  else:
      rec = 0
  
  # Precisão avalia as classificações da classe positivo(vp=1 e fp=0) que o modelo fez e quantas estão corretas.
  if (vp_s+fp_s) != 0:
      pre = (vp_s)/(vp_s+fp_s)
  else:
      pre = 0  

  # F1 é a média harmônica entre precisão e recall.
  if (pre + rec) != 0:  
    f1 = 2 * ((pre * rec)/(pre + rec))
  else:
    f1 = 0

  # Média da perda da avaliação  
  media_test_epoca_loss = np.mean(test_batch_losses)

  del py_input_ids
  del py_attention_masks
  del py_labels
  del test_batch_losses
  del lote_teste_bar
  
  del model
  del tokenizer
  del documentos_teste
  del classes_teste
  del id_documentos_teste

  return media_test_epoca_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, lista_resultado_avaliacao

### Função que salva o resultado da classificação

In [ ]:
def getNomeDiretorioResultados(training_args, model_args):
  
  # Monta o nome do arquivo com parâmetros
  nome_arquivo = ""
  nome_arquivo = nome_arquivo + "P_" + str(model_args.documentos_perturbados)
  nome_arquivo = nome_arquivo + "_K_" + str(model_args.top_k_predicao) 
     
  return nome_arquivo

In [ ]:
# Import das bibliotecas.
import os
import datetime

def salvaResultadoClassificacao(lista_resultado_avaliacao):

  if model_args.salvar_classificacao:

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()
    
    # Nome arquivo resultado
    NOME_ARQUIVO_CLASSIFICACAO = NOME_BASE_SAIDA + getSufixoNomeArquivoSaida(training_args, model_args) + str(model_args.fold) + MODELO_BERT + TAMANHO_BERT 
  
    # Diretório para salvar o arquivo.
    DIRETORIO_CLASSIFICACAO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/kfold/Classificacao/" + getNomeDiretorioResultados(training_args, model_args) + "/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_CLASSIFICACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_CLASSIFICACAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_CLASSIFICACAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_CLASSIFICACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_CLASSIFICACAO_COMPLETO = DIRETORIO_CLASSIFICACAO + NOME_ARQUIVO_CLASSIFICACAO + ".csv"

    # Gera todo o conteúdo a ser salvo no arquivo
    novo_conteudo = ""        
    for resultado in lista_resultado_avaliacao:      
      novo_conteudo = novo_conteudo + data_e_hora.strftime("%d/%m/%Y %H:%M") + ";" + str(resultado[0]) + ";" + str(resultado[1]) + ";" + str(resultado[2]) + "\n"

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO):
      logging.info("Atualizando arquivo classificação: {}.".format(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()

      del conteudo
      del arquivo
      del lista_resultado_avaliacao
      
    else:
      logging.info("Criando arquivo classificação: {}.".format(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_CLASSIFICACAO_COMPLETO,'w')
      arquivo.writelines('data;id;classe;predicao\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

      del arquivo
      del lista_resultado_avaliacao    

### Função que salva o resultado da avaliação

Salva o resultado da avaliação do conjunto de dados de teste do fold especificado.

In [ ]:
# Import das bibliotecas.
import os
import datetime

def salvaResultadoAvaliacao(media_test_loss, acc, rec, pre, f1, vp_s, vn_s, fp_s, fn_s, treinamento_total):

  if model_args.salvar_avaliacao:
    
    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    # Nome arquivo resultado
    NOME_ARQUIVO_AVALIACAO = NOME_BASE_SAIDA + getSufixoNomeArquivoSaida(training_args, model_args) + str(model_args.fold) + MODELO_BERT + TAMANHO_BERT 

    # Diretório para salvar o arquivo.
    DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/kfold/Avaliacao/" + getNomeDiretorioResultados(training_args, model_args) + "/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_AVALIACAO):  
      # Cria o diretório
      os.makedirs(DIRETORIO_AVALIACAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_AVALIACAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_AVALIACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + ".csv"

    # Conteúdo a ser adicionado.
    novo_conteudo = NOME_ARQUIVO_AVALIACAO + ";" +  data_e_hora.strftime("%d/%m/%Y %H:%M") + ";"  + treinamento_total + ";"  + str(acc) + ";"  +  str(vp_s) + ";"  +  str(vn_s) + ";" +  str(fp_s) + ";" +  str(fn_s) + "\n"

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      logging.info("Atualizando arquivo resultado: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)  
      # Fecha o arquivo.
      arquivo.close()

      del conteudo
      del arquivo

    else:
      logging.info("Criando arquivo resultado: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      arquivo.writelines('arquivo;data;tempo;acuracia;vp;vn;fp;fn\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

      del arquivo

### Função que carrega e calcula a média da acurácia dos folds


In [ ]:
# Import das bibliotecas.
import os
import pandas as pd

def carregaResultadoAvaliacao(NOME_ARQUIVO_AVALIACAO):

  NOME_ARQUIVO_EXECUCAO = NOME_ARQUIVO_AVALIACAO + "X" + MODELO_BERT + TAMANHO_BERT

  logging.info("Média dos arquivos: {}".format(NOME_ARQUIVO_EXECUCAO))

  # Diretório para salvar o arquivo.
  DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/kfold/Avaliacao/" + getNomeDiretorioResultados(training_args, model_args) + "/"

  # Verifica se o diretório dos resultados existem.
  if os.path.exists(DIRETORIO_AVALIACAO):
    # Acumuladores.
    soma_acuracia = 0
    listaTempo = []
    conta_folds = 0

    # Percorre os arquivos de resultados.
    for f in range(10):  
      # Nome do arquivo a ser aberto.
      NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + str(f+1) + MODELO_BERT + TAMANHO_BERT + ".csv"    
      # Verifica se o arquivo existe.
      if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
        
        # Carrega os dados do arquivo  
        dados = pd.read_csv(NOME_ARQUIVO_AVALIACAO_COMPLETO, sep=';')
        # Mostra os dados do teste do fold.
        for index, linha in dados.iterrows():
        
          # Cálculo das estatísticas
          acc = (linha['vp']+linha['vn'])/(linha['vp']+linha['vn']+linha['fp']+linha['fn'])
          if (linha['vp']+linha['fn']) != 0:
              rec = (linha['vp'])/(linha['vp']+linha['fn'])
          else:
              rec = 0
          if (linha['vp']+linha['fp']) != 0:
              pre = (linha['vp'])/(linha['vp']+linha['fp'])
          else:  
              pre = 0
          if (pre + rec) != 0:  
              f1 = 2 * ((pre * rec)/(pre + rec))
          else:
              f1 = 0
          qtde_testes = linha['vp']+linha['vn']+linha['fp']+linha['fn']
          logging.info("Arquivo: {}, Data: {}, Tempo: {}, QtdeTeste: {:3d}, Acc: {:.8f}, Rec: {:.8f}, Pre: {:.8f}, F1:{:.8f}, vp: {:4d}; vn: {:4d}; fp: {:4d}; fn: {:4d}".format( 
               linha["arquivo"], linha["data"], linha["tempo"], qtde_testes, acc, rec, pre, f1, linha["vp"], linha["vn"], linha["fp"], linha["fn"]))    
           
          # Guarda o tempo.
          listaTempo.append(str(linha['tempo']))

        # Procura a maior acurácia.
        soma_acuracia = soma_acuracia + dados['acuracia'].max()
        # Conta o número de folds.
        conta_folds = conta_folds + 1

        del dados
    
    # Mostra a soma da acurácia . 
    logging.info("Total acurácia                                       : {:.8f}.".format(soma_acuracia))
    # Mostra a quantidade de folds.
    logging.info("Quantidade de folds                                  : {}.".format(conta_folds))  
    # Calcula a média.
    if conta_folds != 0:
      media = soma_acuracia/conta_folds
      logging.info("A média da acurácia de {:2d} folds é                    : {:.8f}.".format(conta_folds, media))
      logging.info("O tempo gasto na execução do treinamentoa {:2d} folds é : {}.".format(conta_folds, somaTempo(listaTempo)))
      logging.info("A média de tempo de execução de {:2d} folds é           : {}.".format(conta_folds, mediaTempo(listaTempo)))
      # Guarda o resultado da execução
      lista_resultado_execucoes.append(NOME_ARQUIVO_EXECUCAO + ";" + str(media) + ";" + str(somaTempo(listaTempo)))
  else:
    logging.info("Diretório com os resultados não encontrado.")
  
  del listaTempo

## 4.6 Salva e copia o modelo ajustado

### Função que salva o modelo ajustado no wandb


In [ ]:
def salvaModeloWandb(model, model_args):
  
  # Salvando o Modelo para o wandb
  if model_args.use_wandb and model_args.salvar_modelo_wandb:
  
    # Salva o modelo para o wandb    
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'model_dict.pt'))

    logging.info("Modelo salvo no wandb.")

### Função que salva e copia o modelo ajustado

Salva e  modelo e o tokenizador no disco e copia para o google drive.

In [ ]:
# Import de bibliotecas.
import os

def salvaModelo(model, tokenizer, model_args):
  
  if model_args.salvar_modelo:

    # Salvando as melhores práticas: se você usar nomes padrão para o modelo, você pode recarregá-lo usando from_pretrained ()

    # Diretório de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = '/content/modelo_ajustado/' + model_args.fold

    # Cria o diretório de saída se necessário.
    if not os.path.exists(DIRETORIO_LOCAL_MODELO_AJUSTADO):
      os.makedirs(DIRETORIO_LOCAL_MODELO_AJUSTADO)

    logging.info("Salvando o modelo para {}.".format(DIRETORIO_LOCAL_MODELO_AJUSTADO))

    # Salve um modelo treinado, configuração e tokenizer usando `save_pretrained ()`.
    # Eles podem então ser recarregados usando `from_pretrained ()`.
    model_to_save = model.module if hasattr(model, 'module') else model  # Cuide do treinamento distribuído/paralelo
    model_to_save.save_pretrained(DIRETORIO_LOCAL_MODELO_AJUSTADO)
    tokenizer.save_pretrained(DIRETORIO_LOCAL_MODELO_AJUSTADO)

    # Boa prática: salve seus argumentos de treinamento junto com o modelo treinado.
    torch.save(model_args, os.path.join (DIRETORIO_LOCAL_MODELO_AJUSTADO, 'mode_args.bin'))
    torch.save(training_args, os.path.join (DIRETORIO_LOCAL_MODELO_AJUSTADO, 'training_args.bin'))

    logging.info("Modelo salvo.")

    # Para salvar seu modelo nas sessões do Colab Notebook, faça o download no seu computador local ou, idealmente, copie-o no seu Google Drive.
    # Copia o arquivo do modelo para o diretório no Google Drive.
    !cp -r '$DIRETORIO_LOCAL_MODELO_AJUSTADO'* '$DIRETORIO_REMOTO_MODELO_AJUSTADO'

    logging.info("Modelo copiado.")


# 5 Execução do treinamento e avaliação 

Gera o sufixo do nome do arquivo de saída com os parâmetros da execução

Exemplo de formado de sufixo.
`K_1_P_1_E_4_e_1_lr_5_b_8_4_f`
  - P = documentos perturbados
  - K = previsões palavras 
  - E = número total de épocas de treinamento
  - e = número da época executada
  - lr = taxa de aprendizagem
  - b = lotes de treino e avaliação  
  - f = número do fold

In [ ]:
def getSufixoNomeArquivoSaida(training_args, model_args):

  # Recupera o número inteiro da taxa de aprendizagem
  taxa_inteiro = int(training_args.learning_rate*100000)

  # Monta o nome do arquivo com parâmetros
  nome_arquivo = ""
  nome_arquivo = nome_arquivo + "_P_" + str(model_args.documentos_perturbados) 
  nome_arquivo = nome_arquivo + "_K_" + str(model_args.top_k_predicao) 
  nome_arquivo = nome_arquivo + "_E_" + str(training_args.num_train_epochs)
  nome_arquivo = nome_arquivo + "_e_" + str(model_args.epoca)   
  nome_arquivo = nome_arquivo + "_lr_" + str(taxa_inteiro)  
  nome_arquivo = nome_arquivo + "_b_" + str(training_args.per_device_train_batch_size) 
  nome_arquivo = nome_arquivo + "_" + str( training_args.per_device_eval_batch_size) 
  nome_arquivo = nome_arquivo + "_f" 
   
  return nome_arquivo

## 5.1 Função de treinamento e avaliação de todos os folds

Carrega os folds para serem avaliados

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook
import time
import datetime
import gc

def procedimentoTreinamentoAvaliacaoFolds(modelo, taxa_de_aprendizagem):

  # Barra de progresso dos folds
  fold_bar = tqdm_notebook(range(inicio_fold, fim_fold+1), desc=f'Folds ', unit=f'fold', total=fim_fold)

  # Percorre todos os folds do intervalo de inicio_fold até fim_fold
  for fold_i in fold_bar:

    # Seta o parâmetro do fold
    model_args.fold = fold_i

    logging.info("Processamendo do fold: {}.".format(model_args.fold))
    logging.info("   com o modelo: {}".format(modelo))
    logging.info("   até época {} e taxa de aprendizagem {}.".format(training_args.num_train_epochs, taxa_de_aprendizagem))  
    
    # Marca o tempo de início do processamento
    processamento_fold_t0 = time.time()

    # Carregando o modelo   
    model, tokenizer = carregaBERT(model_args)
        
    # Conecta o modelo a GPU
    model = conectaGPU(model, device)

    # Função de carregamento dos dados de um fold
    df_dados_train, df_dados_test = carregamentoDadosFold(model_args.fold)

    # Descartando documentos muito grandes
    df_dados_train, df_dados_test = descarteDocumentosGrandes(tokenizer, model_args.max_seq_len, df_dados_train, df_dados_test)
    
    # Pega as listas de documentos de treino e seus rótulos.
    documentos_treino = df_dados_train.documento.values
    classes_treino = df_dados_train.classe.values
    id_documentos_treino = df_dados_train.id.values

    # Pega as listas de documentos teste e seus rótulos.
    documentos_teste = df_dados_test.documento.values
    classes_teste = df_dados_test.classe.values
    id_documentos_teste = df_dados_test.id.values

    del df_dados_train
    del df_dados_test

    # Mostra o resultado dos dados carregados.
    logging.info("Tamanho do conjunto de dados : {} / Treino: {} / Teste: {}.".format(len(documentos_treino) + len(documentos_teste), len(documentos_treino), len(documentos_teste)))
    
    #################  Treinamento

    # Carrega o otimizador
    optimizer = carregaOtimizador(training_args, model)

    # Carrega o agendador
    scheduler = carregaAgendador(training_args, optimizer, numero_etapas=len(documentos_treino))

    # Registra o tempo inicial.
    treinamento_t0 = time.time()
    
    # Realiza o treinamento.
    realizaTreinamento(model, tokenizer, optimizer, scheduler,
                       documentos_treino, classes_treino, id_documentos_treino, 
                       documentos_teste, classes_teste, id_documentos_teste, training_args.num_train_epochs)
    
    # Medida de quanto tempo levou a execução do treinamento.
    treinamento_f = time.time()
    treinamento_total = formataTempo(treinamento_f - treinamento_t0)  
    logging.info("  Tempo total treinamento       : {:}.".format(treinamento_total))
    
    #################  Treinamento

    # Apaga os dados
    del documentos_treino
    del classes_treino
    del id_documentos_treino
    
    del documentos_teste
    del classes_teste
    del id_documentos_teste

    del optimizer
    del scheduler
    del model
    del tokenizer

    # Pega o tempo atual menos o tempo do início do processamento.
    processamento_fold_f = time.time()
    processamento_fold_total = formataTempo(processamento_fold_f - processamento_fold_t0)    
    logging.info("  Tempo processamento fold: {:} (h:mm:ss)\n".format(processamento_fold_total))    

    # Chama o coletor de lixo para esvaziar a memória
    gc.collect()    

## 5.2 Execução o procedimento de treinamento e avaliação para todos os parâmetros

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook

# Barra de progresso modelos
modelo_bar = tqdm_notebook(enumerate(NOMES_MODELO), desc=f'Modelos ', unit=f'modelo', total=len(NOMES_MODELO))

# Percorre todos os modelos a serem avaliados
for modelo_i, modelo in modelo_bar:

  # Seta o parâmetro do modelo
  model_args.pretrained_model_name_or_path = modelo

  # Barra de progresso das taxas de aprendizagem
  taxa_de_aprendizagem_bar = tqdm_notebook(enumerate(TAXAS_DE_APRENDIZAGEM), desc=f'Taxas de aprendizagem ', unit=f'taxa', total=len(TAXAS_DE_APRENDIZAGEM))

  # Executa o treinamento e avaliação para diversas taxas de aprendizagem
  for taxas_de_aprendizagem_i, taxa_de_aprendizagem in taxa_de_aprendizagem_bar:

    # Atualiza a taxa de aprendizagem da avaliação
    training_args.learning_rate = taxa_de_aprendizagem

    # Marca o tempo de início do processamento dos folds
    processamento_todos_fold_t0 = time.time()

    # Executa o treinamento e avaliacao de todos folds para o modelo e taxa de aprendizagem
    procedimentoTreinamentoAvaliacaoFolds(modelo, taxa_de_aprendizagem)

    # Pega o tempo atual menos o tempo do início do processamento.
    processamento_todos_fold_f = time.time()
    processamento_todos_fold_total = formataTempo(processamento_todos_fold_f - processamento_todos_fold_t0)    
    logging.info("  Tempo processamento de todos os folds: {:} (h:mm:ss)\n".format(processamento_todos_fold_total))    


Modelos :   0%|          | 0/1 [00:00<?, ?modelo/s]

Taxas de aprendizagem :   0%|          | 0/5 [00:00<?, ?taxa/s]

Folds :   0%|          | 0/10 [00:00<?, ?fold/s]

2022-02-25 14:56:33,458 : INFO : Processamendo do fold: 1.
2022-02-25 14:56:33,460 : INFO :    com o modelo: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
2022-02-25 14:56:33,464 : INFO :    até época 4 e taxa de aprendizagem 1e-05.
2022-02-25 14:56:33,908 : INFO : Download do arquivo: bert-large-portuguese-cased_pytorch_checkpoint.zip.


  0%|          | 0.00/1.24G [00:00<?, ?B/s]

...

2022-02-25 16:57:24,774 : INFO :    Média perda(loss) do treinamento da época : 0.49699345.
2022-02-25 16:57:24,777 : INFO :    Tempo de treinamento da época             : 0:00:01.
2022-02-25 16:57:24,779 : INFO :    Tempo parcial processamento               : 0:00:59 (h:mm:ss)
2022-02-25 16:57:24,782 : INFO : Realizando avaliação do fold 10 na época: 4.
2022-02-25 16:57:24,784 : INFO : Criando Lotes Inteligentes de 4 amostras com tamanho de lote 32...
2022-02-25 16:57:24,788 : INFO : Criando lotes de tamanho 32...


Lotes :   0%|          | 0/1 [00:00<?, ?lotes/s]

2022-02-25 16:57:24,902 : INFO :    Avaliação loss: 0.52338219; Acc: 1.00000000; Rec: 1.00000000; Pre: 1.00000000, F1:1.00000000, vp:   2; vn:   2; fp:   0; fn:   0
2022-02-25 16:57:24,905 : INFO :    Acurácia do fold 4 na época 10: 1.00000000.
2022-02-25 16:57:24,907 : INFO :    Média perda(loss) do avaliação da época   : 0.52338219.
2022-02-25 16:57:24,910 : INFO :    Tempo de avaliação da época               : 0:00:00.
2022-02-25 16:57:24,912 : INFO :    Tempo parcial do processamento            : 0:00:59 (h:mm:ss)
2022-02-25 16:57:24,914 : INFO : Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold/Classificacao/P_1_K_1/.
2022-02-25 16:57:24,916 : INFO : Criando arquivo classificação: /content/drive/MyDrive/Colab Notebooks/Data/COHEBERT_MANUAL/validacao_classificacao_palavra/kfold/Classificacao/P_1_K_1/AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_5_b_32_32_f10_BERTimbau_large.csv.
2022-02-25 16:57:24,925 : 

acuracia,▁
do_lower_case,▁
fn,▁
fp,▁
max_seq_len,▁
media_test_epoca_loss,▁
media_train_epoca_loss,▁
output_hidden_states,▁
test_batch_loss,▁
train_batch_loss,█▁
vn,▁


2022-02-25 16:57:29,319 : INFO :   Média perda(loss) treinamento : 0.64309025.
2022-02-25 16:57:29,333 : INFO :   Média perda(loss) avaliação   : 0.66197386.
2022-02-25 16:57:29,342 : INFO :   Tempo total treinamento       : 0:01:03.
2022-02-25 16:57:29,364 : INFO :   Tempo processamento fold: 0:02:42 (h:mm:ss)

2022-02-25 16:57:30,387 : INFO :   Tempo processamento de todos os folds: 0:26:58 (h:mm:ss)



## 5.3 Carregando a acurácia média das execuções

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook

TAXAS_DE_APRENDIZAGEM = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5]

LISTA_EPOCAS = [1, 2, 3, 4]

# Guarda um resumo das execuções
lista_resultado_execucoes = []

# Barra de progresso modelos
modelo_bar = tqdm_notebook(enumerate(NOMES_MODELO), desc=f'Modelos ', unit=f'modelo', total=len(NOMES_MODELO))

# Percorre todos os modelos a serem avaliados
for modelo_i, modelo in modelo_bar:

  # Barra de progresso das taxas de aprendizagem
  taxa_de_aprendizagem_bar = tqdm_notebook(enumerate(TAXAS_DE_APRENDIZAGEM), desc=f'Taxas de aprendizagem ', unit=f'taxa', total=len(TAXAS_DE_APRENDIZAGEM))

  # Executa o treinamento e avaliação para diversas taxas de aprendizagem
  for taxas_de_aprendizagem_i, taxa_de_aprendizagem in taxa_de_aprendizagem_bar:

    # Barra de progresso épocas
    epoca_bar = tqdm_notebook(enumerate(LISTA_EPOCAS), desc=f'Épocas ', unit=f'época', total=len(LISTA_EPOCAS))

    # Percorre todos as épocas a serem avaliadas
    for epoca_i, epoca in epoca_bar:

      logging.info("\n")
      logging.info("Acurácia do modelo: {}".format(modelo))
      logging.info("   com época {} e taxa de treinamento {}.".format(epoca, taxa_de_aprendizagem))

      # Seta o parâmetro do modelo
      model_args.pretrained_model_name_or_path = modelo

      # Seta o parâmetro do modelo
      training_args.learning_rate = taxa_de_aprendizagem

      # Seta o parâmetro do modelo
      model_args.epoca = epoca
   
      # Monta o nome do arquivo de log
      NOME_ARQUIVO_AVALIACAO =  NOME_BASE_SAIDA + getSufixoNomeArquivoSaida(training_args, model_args)
      
      # Carrega o resultado
      carregaResultadoAvaliacao(NOME_ARQUIVO_AVALIACAO)

Modelos :   0%|          | 0/1 [00:00<?, ?modelo/s]

Taxas de aprendizagem :   0%|          | 0/5 [00:00<?, ?taxa/s]

Épocas :   0%|          | 0/4 [00:00<?, ?época/s]

2022-02-25 16:57:30,625 : INFO : 

2022-02-25 16:57:30,628 : INFO : Acurácia do modelo: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
2022-02-25 16:57:30,630 : INFO :    com época 1 e taxa de treinamento 1e-05.
2022-02-25 16:57:30,633 : INFO : Média dos arquivos: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_1_b_32_32_fX_BERTimbau_large
2022-02-25 16:57:30,649 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_1_b_32_32_f1_BERTimbau_large, Data: 25/02/2022 14:58, Tempo: 0:00:14, QtdeTeste:   4, Acc: 0.50000000, Rec: 0.00000000, Pre: 0.00000000, F1:0.00000000, vp:    0; vn:    2; fp:    0; fn:    2
2022-02-25 16:57:30,662 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_1_b_32_32_f2_BERTimbau_large, Data: 25/02/2022 15:00, Tempo: 0:00:08, QtdeTeste:   4, Acc: 0.25000000, Rec: 0.00000000, Pre: 0.00000000, F1:0.00000000, vp:    0; vn:    1; fp:    1;

Épocas :   0%|          | 0/4 [00:00<?, ?época/s]

...

2022-02-25 16:57:32,095 : INFO : 

2022-02-25 16:57:32,100 : INFO : Acurácia do modelo: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
2022-02-25 16:57:32,103 : INFO :    com época 1 e taxa de treinamento 3e-05.
2022-02-25 16:57:32,105 : INFO : Média dos arquivos: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_3_b_32_32_fX_BERTimbau_large
2022-02-25 16:57:32,123 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_3_b_32_32_f1_BERTimbau_large, Data: 25/02/2022 15:42, Tempo: 0:00:11, QtdeTeste:   4, Acc: 0.50000000, Rec: 0.50000000, Pre: 0.50000000, F1:0.50000000, vp:    1; vn:    1; fp:    1; fn:    1
2022-02-25 16:57:32,139 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_3_b_32_32_f2_BERTimbau_large, Data: 25/02/2022 15:44, Tempo: 0:00:11, QtdeTeste:   4, Acc: 0.50000000, Rec: 0.50000000, Pre: 0.50000000, F1:0.50000000, vp:    1; vn:    1; fp:    1;

Épocas :   0%|          | 0/4 [00:00<?, ?época/s]

2022-02-25 16:57:32,751 : INFO : 

2022-02-25 16:57:32,755 : INFO : Acurácia do modelo: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
2022-02-25 16:57:32,756 : INFO :    com época 1 e taxa de treinamento 4e-05.
2022-02-25 16:57:32,758 : INFO : Média dos arquivos: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_4_b_32_32_fX_BERTimbau_large
2022-02-25 16:57:32,772 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_4_b_32_32_f1_BERTimbau_large, Data: 25/02/2022 16:06, Tempo: 0:00:11, QtdeTeste:   4, Acc: 0.50000000, Rec: 0.50000000, Pre: 0.50000000, F1:0.50000000, vp:    1; vn:    1; fp:    1; fn:    1
2022-02-25 16:57:32,794 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_4_b_32_32_f2_BERTimbau_large, Data: 25/02/2022 16:09, Tempo: 0:00:12, QtdeTeste:   4, Acc: 0.50000000, Rec: 1.00000000, Pre: 0.50000000, F1:0.66666667, vp:    2; vn:    0; fp:    2;

Épocas :   0%|          | 0/4 [00:00<?, ?época/s]

2022-02-25 16:57:33,448 : INFO : 

2022-02-25 16:57:33,450 : INFO : Acurácia do modelo: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
2022-02-25 16:57:33,453 : INFO :    com época 1 e taxa de treinamento 5e-05.
2022-02-25 16:57:33,456 : INFO : Média dos arquivos: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_5_b_32_32_fX_BERTimbau_large
2022-02-25 16:57:33,467 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_5_b_32_32_f1_BERTimbau_large, Data: 25/02/2022 16:32, Tempo: 0:00:12, QtdeTeste:   4, Acc: 0.50000000, Rec: 0.00000000, Pre: 0.00000000, F1:0.00000000, vp:    0; vn:    2; fp:    0; fn:    2
2022-02-25 16:57:33,480 : INFO : Arquivo: AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_5_b_32_32_f2_BERTimbau_large, Data: 25/02/2022 16:34, Tempo: 0:00:12, QtdeTeste:   4, Acc: 0.50000000, Rec: 1.00000000, Pre: 0.50000000, F1:0.66666667, vp:    2; vn:    0; fp:    2;

Resumo da execução

Acurácia por época.

In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):  
  if "e_1" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_1_b_32_32_fX_BERTimbau_large;0.4;00:01:52
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_2_b_32_32_fX_BERTimbau_large;0.475;00:01:50
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_3_b_32_32_fX_BERTimbau_large;0.5;00:01:50
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_4_b_32_32_fX_BERTimbau_large;0.55;00:01:57
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_5_b_32_32_fX_BERTimbau_large;0.525;00:02:01


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_e_2" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_1_b_32_32_fX_BERTimbau_large;0.5;00:04:15
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_2_b_32_32_fX_BERTimbau_large;0.425;00:04:20
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_3_b_32_32_fX_BERTimbau_large;0.475;00:04:27
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_4_b_32_32_fX_BERTimbau_large;0.475;00:04:29
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_5_b_32_32_fX_BERTimbau_large;0.575;00:04:32


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_e_3" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_1_b_32_32_fX_BERTimbau_large;0.475;00:06:37
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_2_b_32_32_fX_BERTimbau_large;0.525;00:06:49
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_3_b_32_32_fX_BERTimbau_large;0.475;00:06:57
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_4_b_32_32_fX_BERTimbau_large;0.575;00:07:03
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_5_b_32_32_fX_BERTimbau_large;0.65;00:07:07


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_e_4" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_1_b_32_32_fX_BERTimbau_large;0.5;00:09:00
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_2_b_32_32_fX_BERTimbau_large;0.65;00:09:19
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_3_b_32_32_fX_BERTimbau_large;0.6;00:09:31
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_4_b_32_32_fX_BERTimbau_large;0.575;00:09:38
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_5_b_32_32_fX_BERTimbau_large;0.75;00:09:38


Acurácia por taxa de aprendizagem.

In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_lr_1" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_1_b_32_32_fX_BERTimbau_large;0.4;00:01:52
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_1_b_32_32_fX_BERTimbau_large;0.5;00:04:15
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_1_b_32_32_fX_BERTimbau_large;0.475;00:06:37
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_1_b_32_32_fX_BERTimbau_large;0.5;00:09:00


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_lr_2" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_2_b_32_32_fX_BERTimbau_large;0.475;00:01:50
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_2_b_32_32_fX_BERTimbau_large;0.425;00:04:20
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_2_b_32_32_fX_BERTimbau_large;0.525;00:06:49
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_2_b_32_32_fX_BERTimbau_large;0.65;00:09:19


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_lr_3" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_3_b_32_32_fX_BERTimbau_large;0.5;00:01:50
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_3_b_32_32_fX_BERTimbau_large;0.475;00:04:27
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_3_b_32_32_fX_BERTimbau_large;0.475;00:06:57
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_3_b_32_32_fX_BERTimbau_large;0.6;00:09:31


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_lr_4" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_4_b_32_32_fX_BERTimbau_large;0.55;00:01:57
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_4_b_32_32_fX_BERTimbau_large;0.475;00:04:29
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_4_b_32_32_fX_BERTimbau_large;0.575;00:07:03
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_4_b_32_32_fX_BERTimbau_large;0.575;00:09:38


In [ ]:
for i, linha in enumerate(lista_resultado_execucoes):
  if "_lr_5" in linha:
    print(linha)

AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_1_lr_5_b_32_32_fX_BERTimbau_large;0.525;00:02:01
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_2_lr_5_b_32_32_fX_BERTimbau_large;0.575;00:04:32
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_3_lr_5_b_32_32_fX_BERTimbau_large;0.65;00:07:07
AjusteFinoCohebertManual_v1_C_SB_KF_P_1_K_1_E_4_e_4_lr_5_b_32_32_fX_BERTimbau_large;0.75;00:09:38


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento: {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento: 2:02:14 (h:mm:ss)
